# Project

## Dépendance

In [ ]:
! pip install SPARQLWrapper
! pip install exifread
! pip install pandas
! pip install PIL
! pip install ipywidgets

## Collecte de données

In [ ]:
import pandas as pd
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.
 SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 15"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe

### Téléchargement des images

In [ ]:
import requests
import shutil
import os

def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        # Create 'images' directory if it doesn't exist
        if not os.path.exists("images"):
            os.makedirs("images")

        # Extract the filename from the URL and save the image in 'images' directory
        filename = os.path.join("images", os.path.basename(url))
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

def clear_images_directory():
    # Clear 'images' directory if it exists
    if os.path.exists("images"):
        shutil.rmtree("images")

# Clear 'images' directory before every execution
clear_images_directory()

# Assuming 'dataframe' is a DataFrame containing URLs under column 'image'
# Apply the download_image function to each URL in the 'image' column
dataframe.image.apply(download_image)

### Enregistrement des métadonnées
#### Import

In [ ]:
import os
import json
from PIL import Image, TiffImagePlugin
import PIL.ExifTags

#### Define the directory path and initialize the list to store metadata:

In [ ]:
directory_path = "images"
all_metadata = []

if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist!")
    sys.exit()

#### Define the function to cast data types

In [ ]:
def cast(v):
    if isinstance(v, TiffImagePlugin.IFDRational):
        if v.denominator == 0:
            return None  # Handle division by zero gracefully
        return float(v.numerator) / float(v.denominator)
    elif isinstance(v, tuple):
        return tuple(cast(t) for t in v)
    elif isinstance(v, bytes):
        return v.decode(errors="replace")
    elif isinstance(v, dict):
        for kk, vv in v.items():
            v[kk] = cast(vv)
        return v
    else:
        return v

#### Define the function to get main colors

In [ ]:
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans

def get_main_colors(image_path, num_clusters):
    # Open and resize the image
    imgfile = image_path
    imgfile.thumbnail((100, 100))  # Resize the image to a smaller size
    numarray = np.array(imgfile)  # Convert image to numpy array

    # Flatten the image array
    numarray = numarray.reshape((-1, 3))

    # Sample a subset of pixels (optional)
    np.random.shuffle(numarray)
    num_samples = min(10000, len(numarray))  # Adjust the number of samples if needed
    numarray = numarray[:num_samples]

    # Perform KMeans clustering
    clusters = KMeans(n_clusters=num_clusters, n_init=2)
    clusters.fit(numarray)

    # Get the main colors
    main_colors = clusters.cluster_centers_.astype(int)

    return main_colors.tolist()  # Convert array to list


#### Define the function to get the images properties

In [ ]:
def get_image_metadata(imgfile, hasExif):
    metadata = {}

    # Get image format
    metadata['format'] = imgfile.format

    # Get image size
    metadata['size'] = imgfile.size
    
    # Get 3 main colors
    metadata["main_colors"] = get_main_colors(imgfile,3)
    
    if hasExif:
        exif = dict(imgfile._getexif().items())
        if exif:
            # Get image orientation (landscape, portrait, square, etc.)
            if 274 in exif:
                orientation = exif[274]
                if orientation == 1:
                    metadata['orientation'] = 'Landscape'
                elif orientation == 3:
                    metadata['orientation'] = 'Portrait'
                else:
                    metadata['orientation'] = 'Unknown'
            else:
                metadata['orientation'] = 'Unknown'
    else:
        metadata['orientation'] = 'Unknown'
    return metadata

#### Loop through files in the directory and extract metadata

It will also add as tags the format, the size and the 3 main colors

In [ ]:
import os
import json
import sys
from PIL import Image, TiffImagePlugin
import PIL.ExifTags

directory_path = "images"
all_metadata = []

if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist!")
    sys.exit()

for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)) and filename.lower().endswith((".jpg", ".png")):
        print(f"Found image: {filename}")

        try:
            imgfile = Image.open(os.path.join(directory_path, filename))
            exif_data = imgfile._getexif()
            
            metadata_dict = {}
            
            hasExif = False

            if exif_data:
                hasExif = True
                for k, v in imgfile._getexif().items():
                    if k in PIL.ExifTags.TAGS:
                        v = cast(v)
                        metadata_dict[PIL.ExifTags.TAGS[k]] = v
            else:
                print(f"  - No EXIF data found for {filename}")

            # Get additional image metadata (format, size, orientation)
            image_metadata = get_image_metadata(imgfile, hasExif)
            metadata_dict.update(image_metadata)
            
            all_metadata.append({filename: metadata_dict})
            if not metadata_dict:
                print(f"  - No metadata found for {filename}")

        except Exception as e:
            print(f"  - Error processing {filename}: {e}")

directory_path = "output"
output_file = "directory_metadata.json"

if not os.path.exists(directory_path):
            os.makedirs(directory_path)

output_location = directory_path + "/" + output_file
with open(output_location, "w") as json_file:
    json.dump(all_metadata, json_file, indent=4)

print(f"Directory metadata saved to {output_location} (if no errors occurred)")


## Étiquetage et annotation
### Utilisation des utilisateur pour tagger les images

In [ ]:
import ipywidgets as widgets

from os import listdir

images = []

for file in listdir("./images"):
    if (".png" in file) or (".jpg" in file):
        images.append("./images/" + file)
        
from ipywidgets import GridspecLayout, Image, interact
paths = []

#checkboxes = [widgets.Checkbox(value=False, description='Favorite') for _ in range(len(images))]
tags = [widgets.Text(value='', placeholder='Type something', description='String:', disabled=False) for _ in range (len(images))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=2, n_rows=len(images), width='400px')
for i, (img, tag) in enumerate(zip(images, tags)):
  file = open(img, "rb")
  image = file.read()
  image_widget = widgets.Image(
    value=image,
    format=('png' or 'jpg'),
    width=100,
    height=100,
  )
  layout[i,0] = image_widget
  layout[i, 1] = tag

# Button to get selected images
button = widgets.Button(description="Select")

# Output widget to display selected images
output = widgets.Output()

# Function to get selected images
def get_selected_images(btn):
    global paths
    paths = []
    selected_paths = [images[i] for i, tag in enumerate(tags) if tag.value]
    with output:
        output.clear_output()
        print("Selected Images:")
        for path in selected_paths:
            print(path)
            paths.append(path)
    print(paths)
        

# Link button click event to function
button.on_click(get_selected_images)

# Display the layout and button
display(layout, button, output)


In [ ]:
images